In [1]:
from PIL import Image, ImageOps
import numpy as np
import cv2
import os
import cupy as cp
import matplotlib.pyplot as plt
from ultralytics import YOLO
import glob

In [ ]:
# bboxの学習
model = YOLO('yolov8l.pt')

file = "hukusuu_train"

results = model.train(
    data=f'/home/YOLO/{file}/datasets/config.yaml', 
    epochs=1000, 
    imgsz=640, 
    project=f"/home/YOLO/{file}/datasets", 
    device='mps'   
)

In [ ]:
# segmantationの学習
model = YOLO('yolov8n-seg.pt')

results = model.train(
    data='/home/YOLO/0930_mask/datasets/config.yaml', 
    epochs=1000, 
    imgsz=640, 
    project="/home/YOLO/0930_mask/datasets", 
    device='mps'   
)

In [ ]:
model = YOLO('/home/YOLO/datasets/train4/weights/best.pt')

# Predict the model
# model.predict('/home/data/0410/A/Atest/IMG_1364.JPEG', save = False,save_crop=True, retina_masks = True, conf=0.5)
results = model.predict('/home/data/0410/A/Atest/IMG_1367.JPEG')
for i, r in enumerate(results):
    # Plot results image
    im_bgr = r.plot(masks=True)  # BGR-order numpy array
    im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image

    # Show results to screen (in supported environments)
    # r.show()
    r.show(masks=True,boxes=False)

    # Save results to disk
    # r.save(filename=f"results{i}.jpg")

cropされるとき少しパディングされるので、パディングなしでcropしたい場合は、githubからリポジトリをcloneし、plotting.pyのsave_one_boxのパディングの値を0に変更してから実行する必要がある。→したけど変わらん

resultsにすべて入っているっぽいのでそこからどうにかして一番信頼度高いのとってくる


In [ ]:


# モデルの初期化
model = YOLO('/home/YOLO/datasets/train4/weights/best.pt')

# 予測を行いたいフォルダ
folders = ["/home/data/shiitake_maybeuse/A",
        #    "/home/data_org/B/B",
        #    "/home/data_org/CD/CD"
           ]

# 各フォルダに対してループ
for folder in folders:
    # フォルダ内の全JPEGファイルを取得
    files = glob.glob(f"{folder}/*.JPEG")
    # 各ファイルに対して予測を実行
    for file_path in files:
        filename = os.path.basename(file_path)
        model.predict(file_path,  save=True, retina_masks=False, conf=0.5,name=f"/home/data/yolo_seg/A/{filename}")

In [ ]:
#save mask shiitake 

# model = YOLO('/home/YOLO/mask/datasets/train/weights/best.pt')
model = YOLO('/home/YOLO/0930_mask/datasets/train/weights/best.pt')


folders = [
        #   "/home/data/yolo_crop/A",
        #   "/home/data/yolo_crop/B",
        #   "/home/data/yolo_crop/CD",
          "/home/data/0930/crop"
          ]
for folder in folders:
    files = glob.glob(f"{folder}/*.jpg")
    for file_path in files:
        name = os.path.basename(file_path)
        name = name.replace(".jpg","")
         # 画像を読み込んでサイズを取得
        image = cv2.imread(file_path)
        height, width = image.shape[:2]

        # 読み込んだ画像と同じ大きさの黒い画像を作成
        black_image = np.zeros((height, width), dtype=np.uint8)

        results = model(file_path)  # results list
        if folder == "/home/data/yolo_crop/A":
            for i, r in enumerate(results):
                max_idx = np.argmax(results[0].boxes.conf)
                points = np.array(r.masks.xy[max_idx], dtype=np.int32)
                cv2.fillPoly(black_image, [points], 255)
                cv2.imwrite(f"/home/data/yolo_mask_crop/A/{name}.jpg", black_image)
        elif folder == "/home/data/yolo_crop/B":
            for i, r in enumerate(results):
                max_idx = np.argmax(results[0].boxes.conf)
                points = np.array(r.masks.xy[max_idx], dtype=np.int32)
                cv2.fillPoly(black_image, [points], 255)
                cv2.imwrite(f"/home/data/yolo_mask_crop/B/{name}.jpg", black_image)
        elif folder == "/home/data/yolo_crop/CD":
            for i, r in enumerate(results):
                max_idx = np.argmax(results[0].boxes.conf)
                points = np.array(r.masks.xy[max_idx], dtype=np.int32)
                cv2.fillPoly(black_image, [points], 255)
                cv2.imwrite(f"/home/data/yolo_mask_crop/CD/{name}.jpg", black_image)
        elif folder == "/home/data/0930/crop":
            for i, r in enumerate(results):
                max_idx = np.argmax(results[0].boxes.conf)
                points = np.array(r.masks.xy[max_idx], dtype=np.int32)
                cv2.fillPoly(black_image, [points], 255)
                cv2.imwrite(f"/home/data/0930/mask/{name}.jpg", black_image)

In [5]:
#crop

# YOLOモデルのロード
# model = YOLO('/home/YOLO/bbox2/datasets/train/weights/best.pt')
model = YOLO('/home/YOLO/1008_bbox/datasets/train/weights/best.pt')

date = "1216"
# 処理するフォルダのリスト
folders = [
    f"/home/data/{date}/A",
    f"/home/data/{date}/B",
    f"/home/data/{date}/C",
]

for folder in folders:
    files = glob.glob(f"{folder}/*.JPEG")
    for file_path in files:
        name = os.path.basename(file_path).replace(".JPEG", "")

        # 推論を実行
        results = model(file_path, conf=0.1)  # results list
        if results and len(results[0].boxes.conf) > 0:  # confが空でないことを確認
            for i, r in enumerate(results):
                max_idx = np.argmax(results[0].boxes.conf)
                start_x, start_y, end_x, end_y = map(int, results[0].boxes.xyxy[max_idx])
                clip_img = results[0].orig_img[start_y:end_y, start_x:end_x]

                # 画像を保存
                if folder == f"/home/data/{date}/A":
                    save_path = f"/home/data/{date}/crop/A/{name}.jpg"
                elif folder == f"/home/data/{date}/B":
                    save_path = f"/home/data/{date}/crop/B/{name}.jpg"
                elif folder == f"/home/data/{date}/CD":
                    save_path = f"/home/data/{date}/crop/C/{name}.jpg"
                elif folder == f"/home/data/{date}":
                    save_path = f"/home/data/{date}/crop/{name}.jpg"
                cv2.imwrite(save_path, clip_img)


image 1/1 /home/data/1216/A/IMG_1875.JPEG: 480x640 2 shiitake_bboxs, 23.3ms
Speed: 1.1ms preprocess, 23.3ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/data/1216/A/IMG_1832.JPEG: 480x640 2 shiitake_bboxs, 19.5ms
Speed: 1.1ms preprocess, 19.5ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/data/1216/A/IMG_1833.JPEG: 480x640 3 shiitake_bboxs, 10.6ms
Speed: 1.4ms preprocess, 10.6ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/data/1216/A/IMG_1876.JPEG: 480x640 4 shiitake_bboxs, 10.8ms
Speed: 1.0ms preprocess, 10.8ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/data/1216/A/IMG_1881.JPEG: 480x640 2 shiitake_bboxs, 10.7ms
Speed: 1.0ms preprocess, 10.7ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/data/1216/A/IMG_1865.JPEG: 480x640 3 shiitake_bboxs, 12.9ms
Speed: 1.0ms preprocess, 12.9ms inference, 0.3ms post

In [ ]:
#save crop shiitake test


model = YOLO('/home/YOLO/bbox2/datasets/train/weights/best.pt')
# model = YOLO('/home/YOLO/0917_bbox/datasets/train/weights/best.pt')

path = "/home/data/shiitake_maybeuse/B/IMG_1277.JPEG"
results = model(path)  # results list
name = "1266"
#save_crop:save_dir/cls/file_nameに保存
# Visualize the results
for i, r in enumerate(results):
    
    print("確率:conf")
    print(results[0].boxes.conf)
    print("最大のindex")
    print(np.argmax(results[0].boxes.conf))
    max_idx = np.argmax(results[0].boxes.conf)
    print("bbox:xywh")
    print(results[0].boxes.xyxy)
    print("最大のbbox:xywh")
    print(results[0].boxes.xyxy[max_idx])

    print(results[0].boxes)

    cv2.imwrite("orig_img.jpg", results[0].orig_img)
    start_x, start_y, end_x, end_y = map(int, results[0].boxes.xyxy[max_idx])
    clip_img = results[0].orig_img[start_y:end_y,start_x:end_x]
    cv2.imwrite("clip_img.jpg", clip_img)

    r.show()

In [ ]:
import numpy as np
import cv2

# 640x480の黒画像を作成
black_image = np.zeros((1536, 2048), dtype=np.uint8)

# YOLOモデルの読み込みと推論
# model = YOLO('/home/YOLO/mask/datasets/train/weights/best.pt')
model = YOLO('/home/YOLO/0917_bbox/datasets/train/weights/best.pt')
# model = YOLO('/home/YOLO/bbox2/datasets/train/weights/best.pt')
path = "/home/data/0930/IMG_1620.JPEG"
results = model(path)  # results list

# 結果のマスク座標を取得し、黒画像に白で描写
for i, r in enumerate(results):
    # print(f"Result {i}:")
    # for mask in r.masks.xy:
    #     print(mask)  # マスク座標をプリント
    #     # 座標を整数に変換
    #     points = np.array(mask, dtype=np.int32)
    #     # ポリゴンを白で描写
    #     cv2.fillPoly(black_image, [points], 255)
    
    r.show(boxes=True,masks=True)

# 結果を保存
# cv2.imwrite("masked_image.jpg", black_image)